In [2]:
from bb_behavior import plot
from bb_behavior import db as bbdb
import bb_behavior
import pandas as pd
import os
import sys
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
import datetime

import numpy as np
from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
gaps = c.load('with_coordinates', format = CacheFormat.hdf)

In [4]:
gaps.head()

,bee_id,timestamp,x,y,cam_id,hive_side
0,4,2016-07-30 22:39:30+00:00,131.927600,176.709489,2.0,1
1,4,2016-08-04 00:08:30+00:00,325.306436,149.979205,1.0,0
2,4,2016-08-04 01:07:00+00:00,326.047800,149.909267,1.0,0
3,4,2016-08-05 05:29:30+00:00,118.346444,21.785928,0.0,0
4,4,2016-08-05 23:38:00+00:00,333.792683,67.998315,1.0,0


In [5]:
#Add frame to the df 
gaps_sample = gaps.sample(15)

In [23]:
gaps.sample(15).timestamp[0].timestamp()

1469377260.0

In [11]:
frame_ids = gaps_sample.apply(process_row, axis=1)

In [12]:
gaps_sample = gaps_sample.assign(frame_id=frame_ids).dropna()

In [13]:
in_data = [(row.bee_id, row.frame_id, row.timestamp) for _, row in gaps_sample.iterrows()]

In [15]:
for (bee, frame, timestamp) in in_data:
    name = str(frame)+'_' + str(bee)
    video = plot.plot_bees(bee_ids=[bee], 
                       frame_id=frame, frame_margin=20,
                       path_alpha=None, bt_export=None, # '/home/mi/rrszynka/mnt/janek/caches/Videos/bt_export/'+name+".json",
                       plot_labels = True, plot_markers=True)

    for frame in video._frames:
        frame._title = "\n exit: " + str(timestamp)
    
    video._crop_margin = None
    video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + "t2.mp4")
    
#TODO:
# - get a fullscreen vid? or are the small ones better? 
# - what's the size of fullscreen vs small?

# - how do you get centered videos?
# "if the passed bee is not in the frame, it will be "
# if bt_export is off?

In [11]:
bee

2113

In [19]:
bee = 3006 
frame = 12373576517183002035
timestamp = datetime.datetime(2016, 8, 16, 4, 59, 30)

In [20]:
name = str(frame)+'_' + str(bee)

In [21]:
video = plot.plot_bees(bee_ids=[bee], 
                   frame_id=frame, frame_margin=10,
                   path_alpha=None, bt_export=None,
                   plot_labels = True, plot_markers=True)

In [22]:
for frame in video._frames:
    frame._title = "\ntest"

In [24]:
video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + "-test.mp4")

In [10]:
def process_row(row):
    return get_frame_id_for_bee_id_and_timestamp(row['bee_id'], row['timestamp'])

#TODO: analyze how this works 
def get_frame_id_for_bee_id_and_timestamp(bee_id, dt):
    for cam_id in range(4): 
        frames = bbdb.get_frames(cam_id, dt.timestamp(), dt.timestamp() + 5)
        if len(frames) == 0:
            continue
        detections = bbdb.get_bee_detections(bee_id, frames=frames)
        for d in detections:
            if d is not None:
                return d[1]

In [ ]:
det = c.load("DETECTIONS-2016-08-13_15:00:00", type = CacheType.detections, format = CacheFormat.csv)

In [ ]:
det[det.cam_id==0].head(1)

In [ ]:
samples = []

for cam in range(4):
    samples.append(det[det.cam_id==cam].head(1))